# 12.3. Peach-Koehler Force Problem: Generate Data Repository

In [ ]:
#@title 12.3.1. Import some necessary packages
import numpy as np
import matplotlib.pyplot as plt

# to avoide typing np., we gather some necessary functions as follows:
cos  = np.cos 
sin  = np.sin
pi   = np.pi
sqrt = np.sqrt
acos = np.arccos

> <img src=	"	https://i.ibb.co/ZfbZQT3/15.png	"	width="500"/>

In [ ]:
#@title 12.3.2. Define some problem parameters
d_star               = 3*.249e-9; 
cell_res             = 5e-6;
cell_radius          = (np.sqrt(2)/2)*cell_res;
data_num             = 10000000;

force_original_lb = -1.338688085676038e+09;
force_original_ub = +1.338688085676038e+09;

> <img src=	"	https://i.ibb.co/rFSvHfF/06.png	"	width="750"/>

> <img src=	"	https://i.ibb.co/5B7bXwz/13.png	"	width="750"/>


In [ ]:
#@title 12.3.3 Predefined functions
def fun_fg(a,r,g,t):
  return (cos(3*a - 2*g + 2*t) + cos(a - 2*g + 2*t))/(2*r);

def fun_fgi(a,fg,g,t):
  return (cos(3*a - 2*g + 2*t) + cos(a - 2*g + 2*t))/(2*fg);


In [ ]:
#@title 12.3.4 Predefined functions: conversion, scale, and wrap
def fun_ccs2pcs(x,y):
  r = sqrt(x**2 + y**2)
  a = fun_wrap2pi(np.arctan2(y,x))

  return a,r

def fun_pcs2ccs(a,r):
    x = r*cos(a)
    y = r*sin(a)
    return x,y

def fun_wrapTo2pi(x):
  xwrap = np.remainder(x, 2*pi)
  idx = np.abs(xwrap) > 2*pi
  xwrap[idx] -= 2*pi * np.sign(xwrap[idx]);
  return xwrap

def fun_glb2loc(agb,rgb,tgb,agt,rgt):
    xgt, ygt   = fun_pcs2ccs(agt,rgt)
    xgb, ygb   = fun_pcs2ccs(agb,rgb)
    xrt        = xgt - xgb
    yrt        = ygt - ygb
    xlt        = +xrt*cos(tgb) + yrt*sin(tgb) 
    ylt        = -xrt*sin(tgb) + yrt*cos(tgb)
    alt, rlt   = fun_ccs2pcs(xlt,ylt)
    return alt,rlt

def fun_scale(x,lb=0,ub=1):
    return ((x-np.min(x,axis=0))/(np.max(x,axis=0)-np.min(x,axis=0)))*(ub-lb) + lb

def fun_scaleback(x,lb,ub):
    delta_val = ub-lb
    return x*delta_val + lb

> <img src=	"	https://i.ibb.co/k2VkCdc/10.png	"	width="450"/>

> <img src=	"	https://i.ibb.co/t2VKkBC/11.png	"	width="250"/>

In [ ]:
#@title 12.3.5 Generate unbiased data repository
np.random.seed(30)

# gnerate random data in local PCS
theta_original_i = np.random.rand(data_num,1)*2*np.pi
alpha_original_j = np.random.rand(data_num,1)*2*np.pi
theta_original_j = np.random.rand(data_num,1)*2*np.pi

# generate forces randomly
force_original_g = fun_scale(np.random.rand(data_num,1), force_original_lb, force_original_ub);

# generate radials using inverse force function in local PCS
radia_original_j = fun_fgi(alpha_original_j, force_original_g, theta_original_j, theta_original_i)

# wrap negetive radials (if any)
alpha_original_j[radia_original_j<0] = fun_wrapTo2pi(alpha_original_j[radia_original_j<0] + np.pi)
radia_original_j[radia_original_j<0] = np.abs(radia_original_j[radia_original_j<0])

# control the d_start (or d) condition
ind_gd = radia_original_j >= d_star

# turn vectors to column vectors and only keep the datapoints with rj greater than d_star (or d)
theta_original_i = np.expand_dims(theta_original_i[ind_gd], axis = 1)
alpha_original_j = np.expand_dims(alpha_original_j[ind_gd], axis = 1)
theta_original_j = np.expand_dims(theta_original_j[ind_gd], axis = 1)
radia_original_j = np.expand_dims(radia_original_j[ind_gd], axis = 1)

# no concatenation is needed (we only have one set of radials irrespective of planes)

# use standard logarithm for radials to get a better distribution
radia_logarith_j = np.log10(radia_original_j)

# compute unbiased sigmas
force_ubiasedd_g = fun_fg(alpha_original_j, radia_original_j, theta_original_j, theta_original_i)

data_num         = radia_original_j.shape[0] # reduced 

In [ ]:
#@title 12.3.6 Make some figures
fig, axs = plt.subplots(2, 3,figsize=(15,10))

axs[0, 0].hist(theta_original_i,100);
axs[0, 0].set_title('theta_original_i');

axs[1, 0].hist(theta_original_j,100);
axs[1, 0].set_title('theta_original_j');

axs[0, 1].hist(alpha_original_j,100);
axs[0, 1].set_title('alpha_original_j');

axs[1, 1].hist(radia_logarith_j ,100);
axs[1, 1].set_title('radia_original_j ');

axs[0, 2].hist(1 - radia_logarith_j/np.log10(d_star) ,100);
axs[0, 2].set_title('radia_logarith_j ');

axs[1, 2].hist(force_ubiasedd_g ,100);
axs[1, 2].set_title('force_ubiasedd_g');

filename = 'force_histograms.png'
plt.savefig(filename, dpi=600)


> <img src=	"	https://i.ibb.co/k2VkCdc/10.png	"	width="450"/>

> <img src=	"	https://i.ibb.co/t2VKkBC/11.png	"	width="250"/>

In [ ]:
#@title 12.3.7 Create ML datapoints
# calibrate the data
datain_calibrated        = np.concatenate((theta_original_i/(2*np.pi), theta_original_j/(2*np.pi), alpha_original_j/(2*np.pi), 1 - radia_logarith_j/np.log10(d_star)), axis = 1)
dataou_calibrated        = force_ubiasedd_g/force_original_ub


# 12.4. Peach-Koehler Force Problem: Machine Learning Model

> <img src=	"	https://i.ibb.co/HGnXjnH/09.png	"	width="300"/>

In [ ]:
#@title 12.4.1. Import some necessary packages
import tensorflow as tf
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split


In [ ]:
#@title 12.4.2. Train and test split
RTT = 0.1
RRS = 1

datain_tr_calibrated, datain_te_calibrated, dataou_tr_calibrated, dataou_te_calibrated = train_test_split(datain_calibrated, dataou_calibrated, test_size = RTT, random_state = 42)

In [ ]:
#@title 12.4.3. Define neural network parameters
in_neurons = datain_tr_calibrated.shape[1]
ou_neurons = dataou_tr_calibrated.shape[1]

val_split  = 0.01

hn_neurons = [100,75,50,25,12]

ac_fun     = ['relu', 'relu', 'relu', 'relu', 'linear']

ls_fun     = 'mean_squared_error'
op_val     = 'adam'
it_val     = 20
bt_size    = 512

sh_val     = True # shuffle
vr_val     = 1    # learning to be printed


In [ ]:
#@title 12.4.4. Construct the neural network

net        = tf.keras.models.Sequential() # back-to-back layers of neurons (platform)

net.add( tf.keras.layers.Dense(units=hn_neurons[0], activation=ac_fun[0], input_dim = in_neurons) ) # input layer and the 1st hidden layer
net.add( tf.keras.layers.Dense(units=hn_neurons[1], activation=ac_fun[1]) )                         # 2nd hidden layer
net.add( tf.keras.layers.Dense(units=hn_neurons[2], activation=ac_fun[2]) )                         # 3rd hidden layer
net.add( tf.keras.layers.Dense(units=hn_neurons[3], activation=ac_fun[3]) )                         # 4th hidden layer
net.add( tf.keras.layers.Dense(units=ou_neurons   , activation=ac_fun[4]) )                         # output layer

net.compile(optimizer = op_val, loss = ls_fun) # compile the network

In [ ]:
#@title 12.4.5. Network summary
net.summary()

In [ ]:
#@title 12.4.6. Train the network
history = net.fit(datain_tr_calibrated,
                  dataou_tr_calibrated,
                  epochs           = it_val,
                  batch_size       = bt_size, 
                  verbose          = vr_val,
                  shuffle          = sh_val,
                  validation_split = val_split)

In [ ]:
#@title 12.4.7. Estimate outputs of testing data
dataes_te_calibrated = net.predict(datain_te_calibrated)

In [ ]:
#@title 12.4.8. Plot testing real vs estimated values 
plt.figure(figsize=[7,7])

plt.plot(dataou_te_calibrated.ravel(), dataes_te_calibrated.ravel(), 'b*',markersize = 1)
plt.plot([-1.1,1.1], [-1.1,1.1],'r--',linewidth = 2)

plt.title('Real VS Estimated Forces | Testing', fontsize = 20)
plt.xlabel('Real', fontsize = 20)
plt.ylabel('Estimated', fontsize = 20)
plt.legend(['Output Point','Bisector'], fontsize = 20)


In [ ]:
#@title 12.4.9. Plot iteration vs loss
plt.figure(figsize=[7,7])
plt.plot(history.history['loss'],'-')
plt.plot(history.history['val_loss'],'--')
plt.title('Model Loss', fontsize = 20)
plt.ylabel('Loss', fontsize = 20)
plt.xlabel('Iteration Number', fontsize = 20)
plt.legend(['Validation Loss', 'Testing_Loss'], fontsize = 20)